# Data Import & Preparation

In [1]:
import pandas as pd 

# load data into data frames 
df_mort_country = pd.read_csv("../data/child_mortality_0_5_year_olds_dying_per_1000_born.csv", index_col = 0)
df_population = pd.read_csv("../data/population_total.csv", index_col = 0)
df_income = pd.read_csv("../data/income_per_person_gdppercapita_ppp_inflation_adjusted.csv", index_col = 0)
df_continents =  pd.read_csv("../data/continents.csv", sep = ";")

In [2]:
"""
function to prepare dataframes: 
- ensure columns are all integers
- use 'country' as index for all dataframes
- set 'country' back to column
- melt each df to convert it into long table for seaborn
"""

def prep_df(df, value_name):
    df.columns = df.columns.astype(int)
    df.index.name = "country"
    df = df.reset_index() 
    df = df.melt(id_vars='country', var_name='year', value_name= value_name) 
    return(df)

In [3]:
df_mort_country.head()

,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,469.0,469.0,469.0,469.0,469.0,469.0,470.0,470.0,470.0,470.0,...,12.60,12.40,12.20,12.00,11.80,11.60,11.50,11.30,11.10,11.10
Albania,375.0,375.0,375.0,375.0,375.0,375.0,375.0,375.0,375.0,375.0,...,2.32,2.30,2.27,2.24,2.22,2.19,2.16,2.14,2.11,2.11
Algeria,460.0,460.0,460.0,460.0,460.0,460.0,460.0,460.0,460.0,460.0,...,4.28,4.20,4.14,4.07,4.00,3.94,3.88,3.82,3.76,3.76
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.86,0.84,0.83,0.81,0.80,0.79,0.78,0.77,0.76,0.76
Angola,486.0,486.0,486.0,486.0,486.0,486.0,486.0,486.0,486.0,486.0,...,17.70,17.50,17.30,17.10,17.00,16.80,16.60,16.40,16.30,16.30


In [4]:
df_mort_country = prep_df(df_mort_country, "child mortality per 1000 born")
df_mort_country.head()

,country,year,child mortality per 1000 born
0,Afghanistan,1800,469.0
1,Albania,1800,375.0
2,Algeria,1800,460.0
3,Andorra,1800,NaN
4,Angola,1800,486.0


In [5]:
df_population = prep_df(df_population, "country population")
df_income = prep_df(df_income,"income per person")

In [6]:
"""merge all df's into one df"""

df = df_mort_country.merge(df_continents)
df = df.merge(df_population)
df = df.merge(df_income)

In [7]:
df.head(3)

,country,year,child mortality per 1000 born,continent,country population,income per person
0,Afghanistan,1800,469.0,Asia,3280000,603
1,Afghanistan,1801,469.0,Asia,3280000,603
2,Afghanistan,1802,469.0,Asia,3280000,603


In [8]:
"""create new column showing child mortality as percentage by diving by 10"""

df["child mortality per 1000 born in %"] = df["child mortality per 1000 born"] / 10

In [9]:
"""
- group df by continent and year and aggregate column data 
- set index 'continent' and 'year' back to column
"""

df = df.groupby(["continent", "year"]).agg(
    {"child mortality per 1000 born": "mean", "income per person": "mean",
     "child mortality per 1000 born in %": "mean", "country population": 'sum'}
)
df = df.reset_index()
df.head()

,continent,year,child mortality per 1000 born,income per person,child mortality per 1000 born in %,country population
0,Africa,1800,440.448980,660.224490,44.044898,72859400
1,Africa,1801,440.326531,660.265306,44.032653,72969100
2,Africa,1802,440.204082,660.387755,44.020408,73068800
3,Africa,1803,440.102041,660.204082,44.010204,73190600
4,Africa,1804,439.979592,660.306122,43.997959,73292400


In [10]:
df = df.rename(columns={"child mortality per 1000 born in %":"avg child mortality in %"})

In [11]:
df.head()

,continent,year,child mortality per 1000 born,income per person,avg child mortality in %,country population
0,Africa,1800,440.448980,660.224490,44.044898,72859400
1,Africa,1801,440.326531,660.265306,44.032653,72969100
2,Africa,1802,440.204082,660.387755,44.020408,73068800
3,Africa,1803,440.102041,660.204082,44.010204,73190600
4,Africa,1804,439.979592,660.306122,43.997959,73292400


# Creating an animated plot using plotly

In [12]:
import plotly_express as px

In [13]:
df_plot = df.loc[(df["year"]>1952) & (df["year"]<2020)]

In [41]:
fig = px.scatter(df_plot, x="income per person", y="avg child mortality in %", animation_frame="year", animation_group="continent",
           size="country population", color="continent", hover_name="continent",title="Average Child Mortality vs. Average Income per Person",
           log_x=False, size_max=55, range_x=[1000,40000], range_y=[0,50])

In [45]:
""" 
Adjust animation speed
"""
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 500

In [46]:
fig.show()

In [47]:
import plotly.io as pio

In [57]:
pio.write_html(fig, '../html/gapminder.html')

In [54]:
fig_bar = px.bar(df_plot, x="continent", y="country population", animation_frame="year", animation_group="continent",
        color="continent", hover_name="continent",title="Continent Population over Time",
       range_y=[0,4.401117e+09])

In [55]:
fig_bar.layout.yaxis["title"]["text"] = "Continent Population"

In [56]:
fig_bar.show()

In [29]:
fig_bar.layout.updatemenus[0]

layout.Updatemenu({
    'buttons': [{'args': [None, {'frame': {'duration': 500, 'redraw': True},
                          'mode': 'immediate', 'fromcurrent': True, 'transition':
                          {'duration': 500, 'easing': 'linear'}}],
                 'label': '&#9654;',
                 'method': 'animate'},
                {'args': [[None], {'frame': {'duration': 0, 'redraw': True},
                          'mode': 'immediate', 'fromcurrent': True, 'transition':
                          {'duration': 0, 'easing': 'linear'}}],
                 'label': '&#9724;',
                 'method': 'animate'}],
    'direction': 'left',
    'pad': {'r': 10, 't': 70},
    'showactive': False,
    'type': 'buttons',
    'x': 0.1,
    'xanchor': 'right',
    'y': 0,
    'yanchor': 'top'
})

In [34]:
""" 
Adjust animation speed
"""
fig_bar.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 100